# Integration mittels Monte-Carlo-Simulation

Erweiterung von Teil 1 um eine [Monte-Carlo-Simulation](https://mathepedia.de/Monte-Carlo-Methode.html). 

Hier soll der Bereich (Obere und Untere Grenze) der Simulation entweder manuell gewählt werden oder durch den Algorithmus bestimmt werden.

In [2]:
class Integration:
    def __init__(self, function):
        self.function = function

    def evalFunctionAt(self, x):
        return x

    def untersumme(self, start, end, n):
        sum = 0
        width = abs(start - end) / n
        for i in range(n):
            x = start + i * width
            sum += self.evalFunctionAt(x) * width
        return sum
    
    def obersumme(self, start, end, n):
        sum = 0
        width = abs(start - end) / n
        for i in range(1, n+1):
            x = start + i * width
            sum += self.evalFunctionAt(x) * width
        return sum
    
i = Integration("x^2")
print(i.obersumme(2, 5, 10))
print(i.untersumme(2, 5, 10))

10.95
10.049999999999999


In [22]:
import random
import math

class MCIntegration(Integration):
    def __init__(self, function):
        super().__init__(function)

    #estimatig bounds using "random" sampling
    def estimateBounds(self, start, end, samples=1000):
        xs = [random.uniform(start, end) for _ in range(samples)]
        ys = [self.evalFunctionAt(x) for x in xs]
        return min(ys), max(ys)

    def MCSHitOrMiss(self, start, end, numSamples=10000, fMinManual=None, fMaxManual=None):
        if start >= end:
            raise ValueError("start limit must be smaller than end limit")
        
        # defining limits of sampling-area
        if fMinManual is not None and fMaxManual is not None:
            fMin, fMax = fMinManual, fMaxManual
        else:
            fMin, fMax = self.estimateBounds(start, end)

        # defining rectangle
        rectYMin = min(0, fMin)
        rectYMax = max(0, fMax)
        rectArea = (end - start) * (rectYMax - rectYMin)

        hits = 0

        for _ in range(numSamples):
            x = random.uniform(start, end)
            y = random.uniform(rectYMin, rectYMax)
            f_x = self.evalFunctionAt(x)

            # check if dot is under/above the curve
            if f_x >= 0 and 0 <= y <= f_x:
                hits += 1
            elif f_x < 0 and f_x <= y <= 0:
                hits -= 1  # negativ unter x-Achse

        estimatedIntegral = (hits / numSamples) * rectArea
        return estimatedIntegral

f = lambda x: x**2
mc = MCIntegration(f)

# manual y-limits
resultManual = mc.MCSHitOrMiss(0, 1, numSamples=10000, fMinManual=0, fMaxManual=1)
print(f"manual: {resultManual: .7f}")

# explorative y-limits
resultAuto = mc.MCSHitOrMiss(0, 1, numSamples=10000)
print(f"explorative: {resultAuto: .7f}")

print("\n")

f2 = lambda x: x**3
mc2 = MCIntegration(f2)
resultManual2 = mc.MCSHitOrMiss(-1, 1, numSamples=100000, fMinManual=-1, fMaxManual=1)
print(f"manual: {resultManual2: .7f}")

resultAuto2 = mc.MCSHitOrMiss(-1,1, numSamples=100000)
print(f"explorative: {resultAuto2: .7f}")

print("\n")

f3 = lambda x: math.sin(1/x)
mc3 = MCIntegration(f3)
resultManual3 = mc.MCSHitOrMiss(0, math.pi, numSamples=100000, fMinManual=0, fMaxManual=math.pi)
print(f"manual: {resultManual3: .7f}")

resultAuto3 = mc.MCSHitOrMiss(0, math.pi, numSamples=100000)
print(f"explorative: {resultAuto3: .7f}")

print("\n")

f4 = lambda x: math.e**x
mc4 = MCIntegration(f4)

resultManual4 = mc.MCSHitOrMiss(0, 3, numSamples=100000, fMinManual=0, fMaxManual=150000)
print(f"manual: {resultManual3: .7f}")

resultAuto4 = mc.MCSHitOrMiss(0, 3, numSamples=100000)
print(f"explorative: {resultAuto4: .7f}")


manual:  0.4994000
explorative:  0.4914853


manual:  0.0100000
explorative: -0.0032358


manual:  4.9214782
explorative:  4.9291636


manual:  4.9214782
explorative:  4.5042901


### Ursprung der Monte-Carlo-Simulation
- Entwicklung während des Manhatten-Projektes um die Eigenschaften von Neutronenwanderung durch Strahlungsabschirmung zu untersuchen
- Entwickelt von John von Neumann und Stanislav Ulam

### Anwendung Monte-Carlo-Simulation
- Numerische Lösungen von Integralen (v.a. komplexe Integrale)
- Simulation von dynamischen Prozessen (Wetter, Produktionsabläufe)
- Simulation von Gleichgewischtszuständen (neuronale Netze)
- Statische Untersuchung vn Zufallsverteilungen (erhöhte Genauigkeit der Messfehler bei Experimenten durch häufiges simulieren des Experiments)

<p>MCS wird vor allem bei komplexen Aufgaben, die eigentlich eine hohe Rechenleistung erfordern verwendet</p>

### Buffonsches Nadelexperiment
- dient der Bestimmung von $\pi$
- Wahrscheinlichkeit, dass eine Nadel die Linie kreuzt hängt wegen der Rotationssymmetrie der Nadelorientierung mit $\pi$ zusammen
- $p = \frac{2l}{g\pi} \Rightarrow \pi=\frac{2l}{gp} $
- g ist Abstand der Linien, l ist Länge der Nadeln

![title](img/needles.gif) [1]

### Hit-or-miss oder die direkte MCS?
- Hit-or-miss $$mittlerer Funktionswert \times Intervalllänge$$
- Direkt $$Fläche\,über\,Punktverhältnis\,schätzen$$
test

### Nachteile
- Konvergiert sehr langsam: Ungefährer Fehler bei $$\frac{1}{\sqrt{N}}$$
- Die Schätzung ist mit großer Unsicherheit zu bewerten
- Reproduzierbarkeit nur bedingt gewährleistet (PRNGs daher unerlässlich)

Grundsätzlich gilt, dass analytische und experimentelle Lösungen bevorzugt werden sollten, da sie akurater sind.

### Funktionsweise
- Erstellung von zufälligen Stichproben (gleichmäßig im Intervall verteilt)
- Funktionswert dieser Stichproben berechnen
- Mittelwert dieser Funktionswerte berechen

<p>Manuelle Wahl des Sampling-Bereichs: Angabe der expliziten Grenzen</p>
<p>Explorative Methode: Min und Max innerhalb des Intervalls bestimmen</p>


### Mathematische Aspekte
- direkt $$\int_{a}^b f(x)\,dx \approx\,(b-a) \cdot \frac{1}{N} \sum_{i=1}^N f(x_i)$$ $$mit\,x_i \in [a,b]\,zufällig\,gezogen$$
- hit-or-miss $$A = RA \cdot (hits/all\,points)$$

In [9]:
import numpy as np
# Beispiel-Funktion
def f(x):
    return x**2  # Beispiel: f(x) = x^2

a, b = 0, 1  # Integrationsgrenzen
N = 10000    # Anzahl der Zufallspunkte

xRandom = np.random.uniform(a, b, N)
integralEstimate = (b - a) * np.mean(f(xRandom))
print(f"Monte-Carlo-Schätzwert des Integrals: {integralEstimate}")

Monte-Carlo-Schätzwert des Integrals: 0.33549393774845215


# Quellen
Aus Zotero importieren.
[1] https://faculty.uml.edu/klevasseur/courses/m419/proj/buffon/buffon.html
